In [1]:
import jupyter

# Comparação por scaler
| Scaler               | Train (↓ melhor) | Test (↓ melhor) | Comentário                     |
| -------------------- | ---------------- | --------------- | ------------------------------ |
| **PowerTransformer** | **0.0058**       | **0.0354**      | 🔥 Melhor separabilidade geral |
| QuantileTransformer  | 0.0112           | 0.0380          | Bom em geral                   |
| StandardScaler       | 0.0115           | 0.0375          | Similar ao Quantile            |
| RobustScaler         | 0.0119           | 0.0464          | Um pouco pior em teste         |
| **MinMaxScaler**     | 0.0260           | **0.0664**      | 🚫 Fraca generalização         |
| **MaxAbsScaler**     | **0.0444**       | 0.0524          | 🚫 Muito fraco                 |


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

from tqdm import tqdm
from src.utils.system import boot
from src.defaults import TOP2_STOCK_BY_SECTOR, RANDOM_SEEDS
from src.data.feature_pipeline import load_base_dataframe

DEVICE = boot()
OHLCV_DF = load_base_dataframe()

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
from hurst import compute_Hc
from pyts.image import RecurrencePlot
import datetime

# PROJECT SETUP ==================================
TICKERS         = TOP2_STOCK_BY_SECTOR
#TICKERS = OHLCV_DF['symbol'].unique()
#TICKERS = TOP2_STOCK_BY_SECTOR 

CHAOS_THRESHOLD = 0.45
WINDOW_SIZE     = 120
ENTROPY_BINS    = 10
N_WEEK_DAYS     = 5
STEP_SIZE       = 20
MAX_LEN         = 64
LOOKBACK_BUFFER = 40
ROLLING_WINDOW  = 5
START_DATE      = "2023-01-01",
END_DATE        = "2025-05-01"#pd.Timestamp.today().replace(day=1)
TRAIN_MONTHS    = 2
TEST_MONTHS     = 1
MIN_WINDOW_DAYS = 20
NUM_EPOCHS      = 1
SEEDS = RANDOM_SEEDS[:5]

""" 
DEVELOPER NOTES:
@todo : onehot encode day_of_week so the Model wont use the data as a sized sequence 
"""
BASIC_FEATURES=[
    "close"	,          # Core price for reward and trend awareness
    "volume",          # Volume for activity level
    "candle_body",     # Price strength (close-open)
    "upper_shadow",    # Wick size = volatility / exhaustion
    "lower_shadow",    # Same as above
    "order_flow",      # Flow = pressure indicator (buy/sell imbalance)
    "price_change",    # Price momentum short term
    "volatility",      # Recent price dispersion
    "momentum",        # Rolling price trend
    "vix_norm",        # Implied market risk normalized
    "market_return_1d",# Market regime alignment
    "day_of_week"      # heard it's important                
]

# That’s 12 columns, enough to:
# * See price movement
# * Detect regime shifts
# * Respond to risk

# Second Round:
# * overnight_price_change → if overnight gaps matter to your strategy
# * trade_count_change → intraday activity shifts
# * sp500_norm → macro regime normalization

DEFAULT_INTERNAL_FEATURES = [
    "position",
    "holding_period",
    "cumulative_reward",
    "pct_time",
    "drawdown",
    "unrealized_pnl",
    "entry_price",
    "time_in_position",
    "prev_action"
]



# Data preparation

In [4]:
# STEP 1 - GET DATA =======================
# POC

EXPERIENCE_NAME="regime_detection_and_alignment__v2"

results_path = f"data/experiments/{EXPERIENCE_NAME}.csv"

TICKERS = TOP2_STOCK_BY_SECTOR
FEATURES =  ["volatility", "momentum", "kurtosis", "entropy", "adf_pvalue", "hurst"]
df = OHLCV_DF[OHLCV_DF['symbol'].isin(TICKERS)].copy()


#df = OHLCV_DF[OHLCV_DF['symbol'] == ticker].copy()

df = df.sort_values("date").reset_index(drop=True)




In [25]:
import os
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn.cluster import KMeans
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, MaxAbsScaler, MinMaxScaler, StandardScaler, RobustScaler
from scipy.stats import f_oneway
from statsmodels.tsa.stattools import adfuller
from hurst import compute_Hc
import scipy.stats as stats
import ace_tools_open as tools

import random
import numpy as np

def set_global_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    
# ------------------------
# Regime Indicators (Causal)
# ------------------------
def compute_regime_indicators(df, window=30, hurst_window=128):
    df = df.copy()

    df["volatility"] = df["close"].shift(1).rolling(window).std()
    df["momentum"] = df["close"].pct_change(periods=window).shift(1)
    df["kurtosis"] = df["close"].shift(1).rolling(window).apply(lambda x: stats.kurtosis(x), raw=True)
    df["entropy"] = df["close"].shift(1).rolling(window).apply(
        lambda x: stats.entropy(np.histogram(x, bins=10)[0] + 1), raw=True
    )
    df["adf_pvalue"] = df["close"].shift(1).rolling(window).apply(
        lambda x: adfuller(x)[1] if len(x.dropna()) == window else np.nan
    )

    hurst_vals = []
    for i in range(len(df)):
        if i >= hurst_window:
            window_data = df["close"].iloc[i - hurst_window:i].dropna()
            try:
                hurst_val = compute_Hc(window_data.values, kind="price", simplified=True)[0]
            except:
                hurst_val = np.nan
        else:
            hurst_val = np.nan
        hurst_vals.append(hurst_val)
    df["hurst"] = hurst_vals

    return df

# ------------------------
# Regime Classifier
# ------------------------
def regime_classifier(df_ind, ticker, features, start_date, split_date, end_date, train_months, test_months, n_clusters,  scaler_class, seed=42,verbose=False, plot=False):
    df_clean = df_ind.dropna().copy()
    df_train = df_clean[(df_clean["date"] >= start_date) & (df_clean["date"] < split_date)].copy()
    df_test = df_clean[(df_clean["date"] >= split_date) & (df_clean["date"] <= end_date)].copy()

    if len(df_train) < 10 or len(df_test) < 10:
        return None

    if scaler_class.__name__ == "QuantileTransformer":
        scaler = QuantileTransformer(output_distribution="normal",random_state=seed)
    elif scaler_class.__name__ == "PowerTransformer":
        scaler = PowerTransformer(method="yeo-johnson")
    else:
        scaler = scaler_class()

    X_train = scaler.fit_transform(df_train[features])
    X_test = scaler.transform(df_test[features])

    kmeans = KMeans(n_clusters=n_clusters, random_state=seed)
    df_train["regime"] = kmeans.fit_predict(X_train)
    df_test["regime"] = kmeans.predict(X_test)
    df_test["cluster_dist"] = kmeans.transform(X_test).min(axis=1)

    pvals_test = {}
    significant_count = 0
    for feat in features:
        groups = [df_test[df_test["regime"] == i][feat] for i in range(n_clusters) if (df_test["regime"] == i).shape[0] > 1]
        if len(groups) >= 2:
            _, pval = f_oneway(*groups)
            pvals_test[feat] = pval
            if pval < 0.05:
                significant_count += 1
        else:
            pvals_test[feat] = None

    result = {
        "ticker": ticker,
        "start_date": start_date,
        "split_date": split_date,
        "end_date": end_date,
        "n_clusters": n_clusters,
        "scaler_name": scaler_class.__name__,
        "train_size": len(df_train),
        "test_size": len(df_test),
        "train_months": train_months,
        "test_months": test_months,
        "seed":seed,
        "regime_train_dist": json.dumps(df_train["regime"].value_counts().to_dict(), sort_keys=True),
        "regime_test_dist": json.dumps(df_test["regime"].value_counts().to_dict(), sort_keys=True),
        "anova_significant_features_test": significant_count
    }

    avg_test_pvals = []
    for feat in features:
        result[f"pval_{feat}_test"] = pvals_test.get(feat)
        avg_test_pvals.append(pvals_test.get(feat))
    result["avg_pval_test"] = np.nanmean(avg_test_pvals)

    pvals_train = {}
    significant_count = 0
    for feat in features:
        groups = [df_train[df_train["regime"] == i][feat] for i in range(n_clusters) if (df_train["regime"] == i).shape[0] > 1]
        if len(groups) >= 2:
            _, pval = f_oneway(*groups)
            pvals_train[feat] = pval
            if pval < 0.05:
                significant_count += 1
        else:
            pvals_train[feat] = None

    avg_train_pvals = []
    for feat in features:
        result[f"pval_{feat}_train"] = pvals_train.get(feat)
        avg_train_pvals.append(pvals_train.get(feat))
    result["avg_pval_train"] = np.nanmean(avg_train_pvals)

    return result

# ------------------------
# Discretizer
# ------------------------
def discretize_result(result, thresholds={"anova_significant_features_test": [0, 2, 4, 6], "test_size": [0, 25, 50, 100]}):
    sig = result["anova_significant_features_test"]
    ts = result["test_size"]

    result["anova_strength"] = (
        "low" if sig < thresholds["anova_significant_features_test"][1] else
        "medium" if sig < thresholds["anova_significant_features_test"][2] else "high"
    )

    result["test_size_cat"] = (
        "tiny" if ts < thresholds["test_size"][1] else
        "small" if ts < thresholds["test_size"][2] else
        "medium" if ts < thresholds["test_size"][3] else "large"
    )

    return result

# ------------------------
# Walkforward Regime Classifier
# ------------------------
def walkforward_refime_classifier(
    df, tickers, results_path, features,
    start_date="2023-01-01",
    train_months=5, test_months=5,
    n_clusters=3,
    seeds=[42],
    scalers=[QuantileTransformer, PowerTransformer, MaxAbsScaler, MinMaxScaler, StandardScaler, RobustScaler]
):
    for ticker in tqdm(tickers):
        df_ticker = df[df["symbol"] == ticker].copy().sort_values("date").reset_index(drop=True)
        df_ticker = compute_regime_indicators(df_ticker)

        start = pd.to_datetime(start_date)
        end = df_ticker["date"].max() - relativedelta(months=train_months + test_months)

        if os.path.exists(results_path):
            cached_results = pd.read_csv(results_path)
            cached_results = cached_results.fillna("None")
            cached_results["start_date"] = pd.to_datetime(cached_results["start_date"]).dt.date.astype(str)
            cached_results["split_date"] = pd.to_datetime(cached_results["split_date"]).dt.date.astype(str)
            cached_results["end_date"] = pd.to_datetime(cached_results["end_date"]).dt.date.astype(str)
            monthly_results = cached_results.to_dict("records")
        else:
            monthly_results = []

        while start < end:
            split = start + relativedelta(months=train_months)
            test_end = split + relativedelta(months=test_months)

            for scaler in scalers:
                for seed in seeds:
                    set_global_seed(seed)

                    # Avoid recomputation
                    if any(
                        r["ticker"] == ticker and
                        r["start_date"] == str(start.date()) and
                        r["split_date"] == str(split.date()) and
                        r["end_date"] == str(test_end.date()) and
                        r["n_clusters"] == n_clusters and
                        r["scaler_name"] == scaler.__name__ and
                        int(r.get("seed", -1)) == seed
                        for r in monthly_results
                    ):
                        continue

                    # Run classifier
                    result = regime_classifier(
                        df_ticker, ticker, features,
                        start_date=str(start.date()),
                        split_date=str(split.date()),
                        end_date=str(test_end.date()),
                        train_months=train_months,
                        test_months=test_months,
                        n_clusters=n_clusters,
                        seed=seed,
                        scaler_class=scaler
                    )

                    if result:
                        result = discretize_result(result)
                        monthly_results.append(result)

            start += relativedelta(months=1)

    results_df = pd.DataFrame(monthly_results)
    results_df["avg_pval_test"] = pd.to_numeric(results_df["avg_pval_test"], errors="coerce")

    # Drop rows where avg_pval_test is NaN
    results_df_clean = results_df.dropna(subset=["avg_pval_test"]).copy()
    results_df_clean.to_csv(results_path, index=False)
    tools.display_dataframe_to_user(name="Regime Classification Results", dataframe=results_df_clean)
    return results_df_clean


In [26]:
results_df = walkforward_refime_classifier(df,TOP2_STOCK_BY_SECTOR,results_path,FEATURES,seeds=RANDOM_SEEDS)

  5%|▍         | 1/22 [00:04<01:42,  4.87s/it]C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (103). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (103). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (103). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (103). n_quantiles is set to n_samples.
  warnings.warn

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python

C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_27904\4085374281.py:114: RuntimeWarning: Mean of empty slice
  result["avg_pval_test"] = np.nanmean(avg_test_pvals)
C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_27904\4085374281.py:114: RuntimeWarning: Mean of empty slice
  result["avg_pval_test"] = np.nanmean(avg_test_pvals)
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (106). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (104). n_quantiles is set to n_samples.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_27904\4085374281.py:114: RuntimeWarning: Mean of empty slice
  result["avg_pval_test"] = np.nanmean(avg_test_pvals)
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (104). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the to

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\s

...

C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_27904\4085374281.py:114: RuntimeWarning: Mean of empty slice
  result["avg_pval_test"] = np.nanmean(avg_test_pvals)
C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_27904\4085374281.py:114: RuntimeWarning: Mean of empty slice
  result["avg_pval_test"] = np.nanmean(avg_test_pvals)
C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_27904\4085374281.py:114: Runti

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (105). n_quantiles is set to n_samples.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:3861: DegenerateDataWarning: at least one input has length 0
  warnings.warn(stats.DegenerateDataWarning('at least one input '
C:\Users\Francisco Sá\AppData\Local\Temp\ipykernel_27904\4085374281.py:114: RuntimeWarni

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (106). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (106). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (106). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\sklearn\preprocessing\_data.py:2829: UserWarning: n_quantiles (1000) is greater than the total number of samples (106). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\Francisco Sá\AppData\Roaming\Python

Regime Classification Results


Loading ITables v2.4.0 from the internet... (need help?)


In [73]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2041 entries, 0 to 2399
Data columns (total 30 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ticker                           2041 non-null   object 
 1   start_date                       2041 non-null   object 
 2   split_date                       2041 non-null   object 
 3   end_date                         2041 non-null   object 
 4   n_clusters                       2041 non-null   int64  
 5   scaler_name                      2041 non-null   object 
 6   train_size                       2041 non-null   int64  
 7   test_size                        2041 non-null   int64  
 8   train_months                     2041 non-null   int64  
 9   test_months                      2041 non-null   int64  
 10  seed                             2041 non-null   int64  
 11  regime_train_dist                2041 non-null   object 
 12  regime_test_dist         

In [28]:
results_df[['scaler_name','avg_pval_train','avg_pval_test']].groupby("scaler_name").mean()

,avg_pval_train,avg_pval_test
scaler_name,,
MaxAbsScaler,0.037847,0.039509
MinMaxScaler,0.025108,0.055048
PowerTransformer,0.008558,0.030491
QuantileTransformer,0.012416,0.046022
RobustScaler,0.016129,0.039108
StandardScaler,0.010952,0.029908


In [82]:
agg_df = (
    results_df[results_df['scaler_name'] == "RobustScaler"][
        [
            "anova_significant_features_test",
            "pval_volatility_test", "pval_momentum_test", "pval_kurtosis_test", "pval_entropy_test",
            "pval_adf_pvalue_test", "pval_hurst_test", "avg_pval_test",
            "pval_volatility_train", "pval_momentum_train", "pval_kurtosis_train", "pval_entropy_train",
            "pval_adf_pvalue_train", "pval_hurst_train", "avg_pval_train",
            "ticker", "start_date", "split_date", "end_date"
        ]
    ]
    .groupby(["ticker", "start_date", "split_date", "end_date"])
    .mean(numeric_only=True)  # Make sure pandas doesn't choke on strings
    .reset_index()
)
agg_df

,ticker,start_date,split_date,end_date,anova_significant_features_test,avg_pval_test,pval_volatility_train,pval_momentum_train,pval_kurtosis_train,pval_entropy_train,pval_adf_pvalue_train,pval_hurst_train,avg_pval_train
0,AAPL,2023-01-01,2023-06-01,2023-11-01,5.444444,0.025273,1.281041e-06,1.030954e-20,2.121785e-02,1.072154e-02,2.163439e-01,1.720348e-01,7.005322e-02
1,AAPL,2023-03-01,2023-08-01,2024-01-01,5.000000,0.047439,5.602702e-19,3.445014e-19,3.386246e-16,4.290744e-08,2.068728e-01,3.291852e-01,8.934301e-02
2,AAPL,2023-04-01,2023-09-01,2024-02-01,5.000000,0.019369,3.975162e-20,1.265997e-48,2.927482e-23,2.819826e-26,1.250609e-03,2.069130e-03,5.532899e-04
3,AAPL,2023-05-01,2023-10-01,2024-03-01,4.000000,0.140984,1.236797e-29,7.673073e-36,3.025303e-24,3.782185e-25,3.498734e-03,1.212986e-02,2.604765e-03
4,AAPL,2023-06-01,2023-11-01,2024-04-01,4.500000,0.101629,7.773426e-06,8.069392e-08,5.731007e-07,2.737300e-07,2.261691e-04,6.949835e-13,3.914501e-05
5,AAPL,2023-07-01,2023-12-01,2024-05-01,4.500000,0.059668,7.792492e-16,8.545968e-14,8.259162e-09,4.316692e-10,4.479580e-03,3.221082e-03,1.283445e-03
6,AAPL,2023-08-01,2024-01-01,2024-06-01,4.100000,0.081646,5.517802e-09,9.333896e-18,2.371914e-03,1.356240e-15,1.061782e-05,5.454926e-23,3.970895e-04
7,AAPL,2023-09-01,2024-02-01,2024-07-01,5.100000,0.037444,1.130191e-20,8.365616e-13,1.765376e-07,1.984825e-03,6.571563e-02,5.969854e-08,1.128345e-02
8,AAPL,2023-10-01,2024-03-01,2024-08-01,6.000000,0.001473,1.892829e-25,7.237311e-07,2.047029e-08,1.808739e-02,2.726541e-01,4.670142e-08,4.845705e-02
9,AAPL,2023-11-01,2024-04-01,2024-09-01,5.800000,0.006300,4.545340e-02,8.382432e-04,3.698588e-03,5.219988e-02,1.238729e-01,1.066723e-03,3.785496e-02


# Checkpoint

### What do we have?
We’ve already constructed a powerful modular pipeline:

1. Causal Regime Feature Extractor

    * Uses only past data (shift(1), rolling windows) to avoid leakage.
    * Extracts volatility, kurtosis, entropy, Hurst, ADF, momentum.


2. Walkforward Regime Classifier
    * Tests all scalers and clusterings in a robust out-of-sample way.
    * Tracks ANOVA p-values to check regime separability.
    * Repeats across seeds, months, and scalers.
    * Stores results in CSV for resumability and comparison.

3. Regime Evaluation Metrics
    * Sharpe per regime: Performance of each regime historically.
    * Persistence: How long each regime usually lasts.
    * Predictive power: Average next-day return for each regime (can be expanded).


### Summary:

* BLOCK 1: "Descriptive regimes" — What does this regime mean?
    * Uses: Sharpe ratio, volatility, return std, etc.
    * Helps answer: What has this regime done in the past?

* BLOCK 2: "Forecastable regimes" — Will this regime persist?
    * Uses: Regime duration/persistence stats.
    * Helps answer: Is this regime stable enough to trade or learn on?

* BLOCK 3: "Predictive regimes" — Can this regime predict what comes next?
    * Uses: Future returns, future Sharpe, forward volatility.
    * Helps answer: Does today’s regime tell us about tomorrow’s market?

### Next steps
| Direction                      | Question it answers                                     |  Tools needed                                          |  Outcome                                                         |
| --------------------------------- | ---------------------------------------------------------- | -------------------------------------------------------- | ----------------------------------------------------------------- |
| **1. Meta-Forecast Model**        | “Given the current regime, what is likely to happen next?” | Historical regime stats + ML (regression/classification) | Predict returns, Sharpe, or volatility                            |
| **2. Transition Modeling**        | “What regime is likely to come next?”                      | Markov models or transformer-based transition learning   | Anticipate shifts and prepare trades                              |
| **3. Regime-specific Agents**     | “How should we act in this regime?”                        | Separate RL agents per regime, or conditioned policy     | Specialized strategies that exploit regime behavior               |
| **4. Regime-weighted RL Rewards** | “Should this episode count more?”                          | Modify reward based on regime predictability             | Efficient learning focus on favorable zones                       |
| **5. Meta-feature integration**   | “Can the agent learn what regimes mean on its own?”        | Feed regime labels or embeddings into agent              | Learn policies that adapt to regime without explicit switch logic |


In [71]:
results_df.values

array([['AAPL', '2023-01-01', '2023-06-01', ..., 0.0049638842321327,
        'high', 'large'],
       ['AAPL', '2023-01-01', '2023-06-01', ..., 0.1036963293397756,
        'high', 'large'],
       ['AAPL', '2023-01-01', '2023-06-01', ..., 0.1160409462957987,
        'high', 'large'],
       ...,
       ['XOM', '2024-08-01', '2025-01-01', ..., 1.5316631849790744e-05,
        'high', 'large'],
       ['XOM', '2024-08-01', '2025-01-01', ..., 0.0010072109873669,
        'high', 'large'],
       ['XOM', '2024-08-01', '2025-01-01', ..., 1.734286550612326e-05,
        'high', 'large']], dtype=object)

In [29]:
import pandas as pd
import numpy as np

# Simulated placeholder for demonstration
# In practice, you will load this from your existing regime classification results + OHLCV data
# Let's define the scaffold for the meta-forecast pipeline

def generate_meta_forecast_dataset(df, regime_results_df, forward_window=5):
    """
    Combine regime info with forward return/volatility as labels.

    Parameters:
    - df: Full OHLCV dataset with 'symbol', 'date', 'close' columns
    - regime_results_df: Output from walkforward_refime_classifier, including 'ticker', 'start_date', 'end_date', 'scaler_name', etc.
    - forward_window: Number of days to look ahead for label creation

    Returns:
    - DataFrame with meta-features and target labels
    """
    meta_samples = []

    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values(["symbol", "date"]).reset_index(drop=True)

    for _, row in regime_results_df.iterrows():
        symbol = row["ticker"]
        start = pd.to_datetime(row["split_date"])
        end = pd.to_datetime(row["end_date"])

        df_sub = df[(df["symbol"] == symbol) & (df["date"] >= start) & (df["date"] <= end)].copy()

        if len(df_sub) < forward_window + 1:
            continue

        df_sub = df_sub.sort_values("date")
        df_sub["forward_return"] = df_sub["close"].pct_change(periods=forward_window).shift(-forward_window)
        df_sub["forward_volatility"] = df_sub["close"].pct_change().rolling(forward_window).std().shift(-forward_window)

        for i, r in df_sub.iterrows():
            sample = row.to_dict()  # regime features
            sample["date"] = r["date"]
            sample["symbol"] = symbol
            sample["forward_return"] = r["forward_return"]
            sample["forward_volatility"] = r["forward_volatility"]
            meta_samples.append(sample)

    return pd.DataFrame(meta_samples)





Meta Forecast Dataset


Loading ITables v2.4.0 from the internet... (need help?)


In [69]:
results_df

,ticker,start_date,split_date,end_date,n_clusters,scaler_name,train_size,test_size,train_months,test_months,...,avg_pval_test,pval_volatility_train,pval_momentum_train,pval_kurtosis_train,pval_entropy_train,pval_adf_pvalue_train,pval_hurst_train,avg_pval_train,anova_strength,test_size_cat
0,AAPL,2023-01-01,2023-06-01,2023-11-01,3,QuantileTransformer,103,107,5,5,...,0.000394,1.454540e-07,1.468414e-12,5.422025e-05,3.510592e-14,2.972894e-02,1.536981e-15,0.004964,high,large
1,AAPL,2023-01-01,2023-06-01,2023-11-01,3,QuantileTransformer,103,107,5,5,...,0.031329,3.253601e-15,6.148191e-01,1.081154e-12,7.005692e-16,7.358851e-03,8.244558e-13,0.103696,high,large
2,AAPL,2023-01-01,2023-06-01,2023-11-01,3,QuantileTransformer,103,107,5,5,...,0.018064,6.045788e-23,6.962415e-01,4.739791e-07,1.668211e-12,3.724085e-06,2.286872e-12,0.116041,high,large
11,AAPL,2023-01-01,2023-06-01,2023-11-01,3,PowerTransformer,103,107,5,5,...,0.045936,3.387158e-21,1.638487e-01,6.649731e-07,5.597653e-10,4.727867e-09,2.979614e-16,0.027308,high,large
12,AAPL,2023-01-01,2023-06-01,2023-11-01,3,PowerTransformer,103,107,5,5,...,0.116985,4.823702e-15,4.578907e-09,1.251658e-04,1.804409e-06,3.536078e-06,3.269507e-13,0.000022,high,large
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,XOM,2024-08-01,2025-01-01,2025-06-01,3,RobustScaler,106,102,5,5,...,0.000285,9.805142e-12,5.304878e-03,1.953672e-21,1.922760e-16,2.235134e-11,6.309823e-28,0.000884,high,large
2396,XOM,2024-08-01,2025-01-01,2025-06-01,3,RobustScaler,106,102,5,5,...,0.000246,1.374736e-13,5.355822e-03,1.554636e-22,2.772914e-14,1.267549e-11,6.841583e-28,0.000893,high,large
2397,XOM,2024-08-01,2025-01-01,2025-06-01,3,RobustScaler,106,102,5,5,...,0.000003,1.381940e-16,9.186519e-15,9.057619e-05,1.323601e-06,2.573193e-15,2.940219e-29,0.000015,high,large
2398,XOM,2024-08-01,2025-01-01,2025-06-01,3,RobustScaler,106,102,5,5,...,0.000067,3.869263e-11,6.043266e-03,1.309534e-20,9.867236e-17,2.875914e-12,2.031801e-28,0.001007,high,large


In [83]:
meta_df = generate_meta_forecast_dataset(df,agg_df)
tools.display_dataframe_to_user(name="Meta Forecast Dataset", dataframe=meta_df)

Meta Forecast Dataset


Loading ITables v2.4.0 from the internet... (need help?)


In [98]:
meta_df["forward_sharpe"] = meta_df["forward_return"] / meta_df["forward_volatility"]
meta_df["direction"] = (meta_df["forward_return"] > 0).astype(int)

In [88]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
drop_cols = [
    'avg_pval_test', 
    'anova_significant_features_test',  # if computed on test window, drop it
    'date', 'ticker', 'symbol', 
    'start_date', 'split_date', 'end_date',  # only if not used as categorical/grouping
    # keep forward_return, forward_volatility as y
]
mdf = meta_df.copy()
mdf.drop(columns=drop_cols,inplace=True)
feature_cols = [
    col for col in mdf.columns
    if col.endswith("_train")   # only train-period features
    # optionally add engineered features, but nothing with _test
]
X = mdf.dropna()[[col for col in mdf.columns if col.startswith("pval_") or "avg" in col or "anova" in col]]
#X = pd.get_dummies(X, columns=["anova_strength"], prefix="anova_strength")
#X[X.columns[X.columns.str.startswith("anova_strength_")]] = \
#    X[X.columns[X.columns.str.startswith("anova_strength_")]].astype(int)

y = mdf.dropna()["forward_return"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = XGBRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("R²:", r2_score(y_test, y_pred))

Index(['ticker', 'start_date', 'split_date', 'end_date',
       'anova_significant_features_test', 'avg_pval_test',
       'pval_volatility_train', 'pval_momentum_train', 'pval_kurtosis_train',
       'pval_entropy_train', 'pval_adf_pvalue_train', 'pval_hurst_train',
       'avg_pval_train', 'date', 'symbol', 'forward_return',
       'forward_volatility'],
      dtype='object')
R²: -0.0029510828090508756


In [109]:
import shap
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor,XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import accuracy_score
scalers=[QuantileTransformer, PowerTransformer, MaxAbsScaler, MinMaxScaler, StandardScaler, RobustScaler]
# Placeholder: replace with actual DataFrame you have
# meta_df = generate_meta_forecast_dataset(...)

prediction_results = []
for seed in RANDOM_SEEDS:
    set_global_seed(seed)
    for scaler_class in tqdm(scalers):
        for target in ["forward_sharpe","forward_return","forward_volatility","direction"]:
        # Simulate meta_df for now to allow code testing (remove if you have real data)
            drop_cols = [
                'avg_pval_test', 
                'anova_significant_features_test',  # if computed on test window, drop it
                'date', 'ticker', 'symbol', 
                'start_date', 'split_date', 'end_date',  # only if not used as categorical/grouping
                # keep forward_return, forward_volatility as y
            ]
            mdf = meta_df.copy()
            mdf.drop(columns=drop_cols,inplace=True)
            feature_cols = [
                col for col in mdf.columns
                if col.endswith("_train")   # only train-period features
                # optionally add engineered features, but nothing with _test
            ]

            X = mdf.dropna()[[col for col in mdf.columns if col.startswith("pval_") or "avg" in col or "anova" in col]]
            #X = pd.get_dummies(X, columns=["anova_strength"], prefix="anova_strength")
            #X[X.columns[X.columns.str.startswith("anova_strength_")]] = \
            #    X[X.columns[X.columns.str.startswith("anova_strength_")]].astype(int)

            y = mdf.dropna()[target]

            # === Train-Test Split
            X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.3)
            # === Scaling ===
            if scaler_class.__name__ == "QuantileTransformer":
                scaler = QuantileTransformer(output_distribution="normal",random_state=seed)
            elif scaler_class.__name__ == "PowerTransformer":
                scaler = PowerTransformer(method="yeo-johnson")
            else:
                scaler = scaler_class()

            X_train_scaled = scaler.fit_transform(X_train)   # Fit only on training
            X_test_scaled = scaler.transform(X_test)         # Transform test

            # === Model Training ===
            if target =="direction":
                
                model = XGBClassifier(use_label_encoder=False, eval_metric="logloss",n_estimators=100, max_depth=3, random_state=seed)
            else:
                model = XGBRegressor(n_estimators=100, max_depth=3, random_state=seed)
            model.fit(X_train_scaled, y_train)

            # === Prediction ===
            y_pred = model.predict(X_test_scaled)

            # === Evaluation ===
            r2 = r2_score(y_test, y_pred)
            rmse = mean_squared_error(y_test, y_pred)
            accuracy = 0
            if target =="direction":
                accuracy = accuracy_score(y_test, (y_pred > 0.5).astype(int))
            print(f"Direction Accuracy: {accuracy:.2%}")
            """
            # === SHAP Analysis ===
            explainer = shap.Explainer(model)
            shap_values = explainer(X_train_scaled)

            # === Visual Output ===
            tools.display_dataframe_to_user(name="Prediction vs Truth", dataframe=pd.DataFrame({
                "true_return": y_test.values,
                "predicted_return": y_pred
            }))

            plt.figure()
            shap.plots.beeswarm(shap_values, max_display=10, show=False)
            plt.title("SHAP values (train set)")
            plt.show()
            """
            #accuracy = (np.sign(y_pred) == np.sign(y_true)).mean()
            print(f"R² Score: {r2:.4f}")
            print(f"RMSE: {rmse:.5f}")
            prediction_results.append({"seed":seed,"r2":r2,"rmse":rmse,"scaler_name":scaler_class.__name__,"accuracy":accuracy,"target":target})
        
prediction_df = pd.DataFrame(prediction_results)
prediction_df


  0%|          | 0/6 [00:00<?, ?it/s]C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [01:25:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
 17%|█▋        | 1/6 [00:00<00:01,  3.97it/s]

Direction Accuracy: 0.00%
R² Score: -0.0699
RMSE: 13.72295
Direction Accuracy: 0.00%
R² Score: -0.0498
RMSE: 0.00093
Direction Accuracy: 0.00%
R² Score: -0.0544
RMSE: 0.00004
Direction Accuracy: 44.91%
R² Score: -1.2150
RMSE: 0.55088


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [01:25:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
 33%|███▎      | 2/6 [00:00<00:01,  3.93it/s]

Direction Accuracy: 0.00%
R² Score: -0.0737
RMSE: 13.77221
Direction Accuracy: 0.00%
R² Score: -0.0475
RMSE: 0.00093
Direction Accuracy: 0.00%
R² Score: -0.3342
RMSE: 0.00005
Direction Accuracy: 46.23%
R² Score: -1.1621
RMSE: 0.53772


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [01:25:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
 50%|█████     | 3/6 [00:00<00:00,  4.83it/s]

...

C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [01:25:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
 67%|██████▋   | 4/6 [00:00<00:00,  5.15it/s]

Direction Accuracy: 0.00%
R² Score: -0.0661
RMSE: 13.67404
Direction Accuracy: 0.00%
R² Score: -0.0399
RMSE: 0.00093
Direction Accuracy: 0.00%
R² Score: -0.1320
RMSE: 0.00004
Direction Accuracy: 47.37%
R² Score: -1.1162
RMSE: 0.52632
Direction Accuracy: 0.00%
R² Score: -0.0733
RMSE: 13.76648
Direction Accuracy: 0.00%
R² Score: -0.0470
RMSE: 0.00093


C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [01:25:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
 83%|████████▎ | 5/6 [00:01<00:00,  5.70it/s]C:\Users\Francisco Sá\AppData\Roaming\Python\Python311\site-packages\xgboost\training.py:183: UserWarning: [01:25:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
100%|██████████| 6/6 [00:01<00:00,  5.13it/s]

Direction Accuracy: 0.00%
R² Score: -0.3350
RMSE: 0.00005
Direction Accuracy: 46.23%
R² Score: -1.1621
RMSE: 0.53772
Direction Accuracy: 0.00%
R² Score: -0.0675
RMSE: 13.69193
Direction Accuracy: 0.00%
R² Score: -0.0579
RMSE: 0.00094
Direction Accuracy: 0.00%
R² Score: -0.0637
RMSE: 0.00004
Direction Accuracy: 44.91%
R² Score: -1.2150
RMSE: 0.55088


,seed,r2,rmse,scaler_name,accuracy,target
0,66923877,-0.069902,13.722954,QuantileTransformer,0.000000,forward_sharpe
1,66923877,-0.049821,0.000934,QuantileTransformer,0.000000,forward_return
2,66923877,-0.054405,0.000038,QuantileTransformer,0.000000,forward_volatility
3,66923877,-1.214969,0.550877,QuantileTransformer,0.449123,direction
4,66923877,-0.073743,13.772215,PowerTransformer,0.000000,forward_sharpe
...,...,...,...,...,...,...
235,81048948,-1.162063,0.537719,StandardScaler,0.462281,direction
236,81048948,-0.067483,13.691931,RobustScaler,0.000000,forward_sharpe
237,81048948,-0.057896,0.000942,RobustScaler,0.000000,forward_return
238,81048948,-0.063666,0.000039,RobustScaler,0.000000,forward_volatility


In [110]:
prediction_df.groupby(['target','scaler_name'])[['r2','rmse','accuracy']].mean()

r2       rmse  accuracy
target             scaler_name                                       
direction          MaxAbsScaler        -1.116212   0.526316  0.473684
                   MinMaxScaler        -1.116212   0.526316  0.473684
                   PowerTransformer    -1.162063   0.537719  0.462281
                   QuantileTransformer -1.214969   0.550877  0.449123
                   RobustScaler        -1.214969   0.550877  0.449123
                   StandardScaler      -1.162063   0.537719  0.462281
forward_return     MaxAbsScaler        -0.039870   0.000926  0.000000
                   MinMaxScaler        -0.039870   0.000926  0.000000
                   PowerTransformer    -0.047466   0.000932  0.000000
                   QuantileTransformer -0.049821   0.000934  0.000000
                   RobustScaler        -0.057896   0.000942  0.000000
                   StandardScaler      -0.046972   0.000932  0.000000
forward_sharpe     MaxAbsScaler        -0.066088  13.674037  0.000000
                   MinMaxScaler        -0.066088  13.674037  0.000000
                   PowerTransformer    -0.073743  13.772215  0.000000
                   QuantileTransformer -0.069902  13.722954  0.000000
                   RobustScaler        -0.067483  13.691931  0.000000
                   StandardScaler      -0.073296  13.766483  0.000000
forward_volatility MaxAbsScaler        -0.132031   0.000041  0.000000
                   MinMaxScaler        -0.132031   0.000041  0.000000
                   PowerTransformer    -0.334230   0.000049  0.000000
                   QuantileTransformer -0.054405   0.000038  0.000000
                   RobustScaler        -0.063666   0.000039  0.000000
                   StandardScaler      -0.334997   0.000049  0.000000

In [65]:
meta_df.iloc[:100].to_csv('analyse_this.csv')

In [96]:
[[col for col in mdf.columns if col.startswith("pval_") or "avg" in col or "anova" in col]]

[['pval_volatility_train',
  'pval_momentum_train',
  'pval_kurtosis_train',
  'pval_entropy_train',
  'pval_adf_pvalue_train',
  'pval_hurst_train',
  'avg_pval_train']]

In [ ]:
feature_cols

In [44]:
""""""
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder( handle_unknown="ignore")
encoded = encoder.fit_transform(X[["anova_strength"]])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(["anova_strength"]))
#df_encoded = pd.concat([X, encoded_df], axis=1)
#df_encoded
encoded_df

ValueError: Shape of passed values is (203851, 1), indices imply (203851, 3)

In [49]:
df_onehot = pd.get_dummies(X, columns=["anova_strength"], prefix="anova_strength")
df_onehot[df_onehot.columns[df_onehot.columns.str.startswith("anova_strength_")]] = \
    df_onehot[df_onehot.columns[df_onehot.columns.str.startswith("anova_strength_")]].astype(int)
df_onehot

,anova_significant_features_test,pval_volatility_test,pval_momentum_test,pval_kurtosis_test,pval_entropy_test,pval_adf_pvalue_test,pval_hurst_test,avg_pval_test,pval_volatility_train,pval_momentum_train,pval_kurtosis_train,pval_entropy_train,pval_adf_pvalue_train,pval_hurst_train,avg_pval_train,anova_strength_high,anova_strength_low,anova_strength_medium
0,6,1.129294e-17,1.419024e-03,1.149730e-05,3.998327e-05,0.000894,3.243969e-09,0.000394,1.454540e-07,1.468414e-12,5.422025e-05,3.510592e-14,2.972894e-02,1.536981e-15,0.004964,1,0,0
1,6,1.129294e-17,1.419024e-03,1.149730e-05,3.998327e-05,0.000894,3.243969e-09,0.000394,1.454540e-07,1.468414e-12,5.422025e-05,3.510592e-14,2.972894e-02,1.536981e-15,0.004964,1,0,0
2,6,1.129294e-17,1.419024e-03,1.149730e-05,3.998327e-05,0.000894,3.243969e-09,0.000394,1.454540e-07,1.468414e-12,5.422025e-05,3.510592e-14,2.972894e-02,1.536981e-15,0.004964,1,0,0
3,6,1.129294e-17,1.419024e-03,1.149730e-05,3.998327e-05,0.000894,3.243969e-09,0.000394,1.454540e-07,1.468414e-12,5.422025e-05,3.510592e-14,2.972894e-02,1.536981e-15,0.004964,1,0,0
4,6,1.129294e-17,1.419024e-03,1.149730e-05,3.998327e-05,0.000894,3.243969e-09,0.000394,1.454540e-07,1.468414e-12,5.422025e-05,3.510592e-14,2.972894e-02,1.536981e-15,0.004964,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214046,6,1.000633e-19,3.217272e-13,7.889114e-20,7.278071e-16,0.000743,4.496235e-08,0.000124,1.039225e-04,2.929665e-18,1.366010e-21,4.700154e-22,8.579669e-08,4.894159e-08,0.000017,1,0,0
214047,6,1.000633e-19,3.217272e-13,7.889114e-20,7.278071e-16,0.000743,4.496235e-08,0.000124,1.039225e-04,2.929665e-18,1.366010e-21,4.700154e-22,8.579669e-08,4.894159e-08,0.000017,1,0,0
214048,6,1.000633e-19,3.217272e-13,7.889114e-20,7.278071e-16,0.000743,4.496235e-08,0.000124,1.039225e-04,2.929665e-18,1.366010e-21,4.700154e-22,8.579669e-08,4.894159e-08,0.000017,1,0,0
214049,6,1.000633e-19,3.217272e-13,7.889114e-20,7.278071e-16,0.000743,4.496235e-08,0.000124,1.039225e-04,2.929665e-18,1.366010e-21,4.700154e-22,8.579669e-08,4.894159e-08,0.000017,1,0,0


In [51]:
df_onehot['anova_strength']

KeyError: 'anova_strength'

In [ ]:
def walkforward_dataset_splits(
    df, tickers, results_path, features,
    start_date="2023-01-01",
    train_months=5, 
    test_months=5,
    
):
    for ticker in tqdm(tickers):
        df_ticker = df[df["symbol"] == ticker].copy().sort_values("date").reset_index(drop=True)
        df_ticker = compute_regime_indicators(df_ticker)

        start = pd.to_datetime(start_date)
        end = df_ticker["date"].max() - relativedelta(months=train_months + test_months)

        
        datasets = []

        while start < end:
            split = start + relativedelta(months=train_months)
            test_end = split + relativedelta(months=test_months)

            train_df = df_ticker[(df_ticker['date']>=start) & (df_ticker['date']<split)]
            test_df = df_ticker[(df_ticker['date']>=split) & (df_ticker['date']<end)]

            datasets.append({
                "start_date":start,
                "split_date": split,
                "end_date": end,
                "train":train_df,
                "test":test_df
            })   

            start += relativedelta(months=1)

    return datasets
